In [1]:
import pandas

items = pandas.read_csv('items.csv', encoding='windows-1251')
storehouse = pandas.read_csv('storehouse_available.csv', encoding='windows-1251')

c:\users\ilya sychev\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
series = dict()

def add_row(row):
    category = row['Товарная иерархия (уровень 2) Наименование']
    if category not in series:
        series[category] = dict()
    item_id = row['Сырье/Товар ID']
    if item_id not in series[category]:
        series[category][item_id] = []
    series[category][item_id].append([row[12], float(row[14].replace(',', '.')) / (float(row[13].replace(',', '.')) + 0.01)])
    
for i in range(items.shape[0]):
    add_row(items.iloc[i])
        

In [17]:
print(series.keys())

dict_keys(['Стирка и уход за бельем', 'Мясо, птица', 'Яйцо', 'Уборка и уход за домом', 'Не присвоено', 'Аксессуары для быт.техники, электроники', 'Бытовая химия для детей', 'Товары для уборки', 'Уход за обувью и изделиями из кожи', 'Средства для борьбы с насекомыми', 'Товары для дома', 'Уход за телом', 'Личная гигиена', 'Дачные туалеты, биоактиваторы', 'Товары для кухни', 'Уход за автомобилем', 'Здоровье и безопасность', 'Мясная гастрономия', 'Украшение дома, сувениры'])


In [21]:
print(series['Яйцо'].keys())

dict_keys([3186437, 3203662, 3186418, 3208367, 3243291, 3134391, 3243282, 3134366, 3243592, 3243561, 3194772, 3243294, 3220173, 3243285, 3220156, 3145207, 3243253, 3148622, 3258876, 3259887, 3279960, 3279922, 3279959, 3279955, 3279956, 3285564, 3285563, 3285565, 3286537, 3286538, 3286410, 3290582, 3302719, 3302906, 3303026, 3303027, 3303025, 3304161, 3308372, 3319498, 3326439, 3326438, 3343354, 3343353, 3326440, 3343355])


In [21]:
print(series['Яйцо'][3186437][-1])

['2016-08-25', 69.72568578553617]


In [7]:
print(series2['Яйцо'][3186437].keys())

dict_keys(['2015-01-31', '2015-01-06', '2015-01-05', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-21', '2015-01-17', '2015-02-05', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-02-01', '2015-02-04', '2015-02-02', '2015-02-03', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-10', '2015-02-11', '2015-02-09', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-24', '2015-02-23', '2015-02-22', '2015-03-02', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-02-20', '2015-02-21', '2015-03-01', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-

In [11]:
# now we are thinking only about number of items bought, fuck promos

In [33]:
from dateutil.rrule import rrule, DAILY
from datetime import datetime
from copy import deepcopy
series_interm = deepcopy(series)
for category in series:
    for item_id in series[category]:
        good_item_id = int(item_id)
        del series_interm[category][item_id]
        series_interm[category][good_item_id] = dict()
        for day in series[category][item_id]:
            series_interm[category][good_item_id][day[0]] = day[1]
            
start_date = datetime(2015, 1, 3)
end_date = datetime(2018, 1, 3)
dates = list(rrule(freq=DAILY, dtstart=start_date, until=end_date))
str_dates = [date.strftime('%Y-%m-%d') for date in dates]
for category in series_interm:
    for item_id in series_interm[category]:
        for i in range(len(str_dates)):
            if str_dates[i] not in series_interm[category][item_id]:
                if i > 0:
                    series_interm[category][item_id][str_dates[i]] = series_interm[category][item_id][str_dates[i - 1]]
                else:
                    series_interm[category][item_id][str_dates[i]] = 0.0

In [35]:
import json
for category in series_interm:
    with open('prices_' + category + '.json', 'w') as f:
        json.dump(series4[category], f)

In [34]:
from dateutil.rrule import rrule, DAILY
from datetime import datetime
series4 = deepcopy(series_interm)
start_date = datetime(2015, 1, 3)
end_date = datetime(2018, 1, 3)
dates = list(rrule(freq=DAILY, dtstart=start_date, until=end_date))
str_dates = [date.strftime('%Y-%m-%d') for date in dates]
for category in series_interm:
    for item_id in series_interm[category]:
        good_item_id = int(item_id)
        del series4[category][item_id]
        series4[category][good_item_id] = []
        for date in str_dates:
            if date not in series_interm[category][item_id]:
                series4[category][good_item_id].append((date, 0.0))
            else:
                series4[category][good_item_id].append((date, series_interm[category][item_id][date]))

In [54]:
import matplotlib.pyplot

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go

layout = go.Layout(
    autosize=False,
    width=1500,
    height=1000,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor='#7f7f7f',
    plot_bgcolor='#c7c7c7',
    showlegend=True
)

def sum_of_item(group, item):
    return sum([q for (date, q) in group[item]])


def make_pandas_series(group, item):
    rng = pandas.date_range(start = start_date, end=end_date, freq='D')
    ts = pandas.Series([q for (date, q) in group[item]], index=rng)
    return ts

def top_k(group_prices: object, group_sales: object, k: object, group_name: object) -> object:
    group_sales_total = [[sum_of_item(group_sales, item), item] for item in group_sales]
    group_sales_total.sort(reverse=True)
    for i in range(min(k, len(group_sales_total))):
        ts = make_pandas_series(group_prices, group_sales_total[i][-1])
        ts.plot(label=group_sales_total[i][-1])
    
    fig = matplotlib.pyplot.gcf()
    plotly_fig = tls.mpl_to_plotly( fig)
    plotly_fig['layout']['width'] = 1500
    plotly_fig['layout']['height'] = 1000
    py.plot(plotly_fig, filename=group_name)


In [43]:
eggs = json.load(open('Яйцо.json'))
eggs_prices = json.load(open('prices_Яйцо.json'))

In [57]:
top_k(eggs_prices, eggs, 5, 'Яйца_цены')

In [58]:
top_k(eggs, eggs, 5, 'Яйца_продажи')

In [59]:
eggs = json.load(open('Мясо, птица.json'))
eggs_prices = json.load(open('prices_Мясо, птица.json'))
top_k(eggs_prices, eggs, 5, 'Мясо, птица_цены')
top_k(eggs, eggs, 5, 'Мясо, птица_продажи')

In [60]:
def top_k_cool(k, name):
        eggs = json.load(open(name+'.json'))
        eggs_prices = json.load(open('prices_'+ name+'.json'))
        top_k(eggs_prices, eggs, k, name+'_цены')
        top_k(eggs, eggs, k, name+'_продажи')
        
        
top_k_cool(3, 'Мясная гастрономия')

In [61]:
top_k_cool(2, 'Товары для дома')

In [62]:
top_k_cool(3, 'Стирка и уход за бельем')

In [63]:
top_k_cool(3, 'Уборка и уход за домом')